# i am using github models

In [6]:
from typing import Optional, Any, List, Dict
from mcp import ClientSession, StdioServerParameters
from contextlib import AsyncExitStack
from openai import OpenAI
from mcp.client.stdio import stdio_client
import asyncio
import nest_asyncio
import json
import os
from dotenv import load_dotenv
load_dotenv('.env')
nest_asyncio.apply()

In [7]:
MODEL = os.getenv("GITHUB_MODEL_NAME")
GUTHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")

In [15]:
class MCPOpenAIClient:
    """A client for interacting with the OpenAI API using the Model Context Protocol (MCP) tools."""

    def __init__(self, model: str = MODEL):
        """initilize the openai mcp client"""
        self.model = model
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
        self.openai_client = OpenAI(
            base_url=GITHUB_ENDPOINT,
            api_key=GUTHUB_TOKEN,
        )
        self.stdio: Optional[Any] = None
        self.write: Optional[Any] = None
        self.token = GUTHUB_TOKEN
        self.endpoint = GITHUB_ENDPOINT

    async def connect_to_server(self, server_path: str = 'server.py'):
        """Connect to the MCP server."""

        # server config
        server_params = StdioServerParameters(
            command='python',
            args=[server_path],
        )

        # connect to the server
        stdio_transport = await self.exit_stack.enter_async_context(
            stdio_client(server_params)
        )
        self.stdio, self.write = stdio_transport
        self.session = await self.exit_stack.enter_async_context(
            ClientSession(self.stdio, self.write)
        )

        # init the connection
        await self.session.initialize()

        # list the available tools
        tools_result = await self.session.list_tools()
        print("\nConnected to server with tools:")
        for tool in tools_result.tools:
            print(f"  - {tool.name}: {tool.description}")

    async def get_mcp_tools(self) -> List[Dict[str, Any]]:
        """Get the list of tools available in the MCP session."""
        if not self.session:
            raise RuntimeError("Session is not initialized. Call connect_to_server first.")
        
        tools_result = await self.session.list_tools()

        return [
            {
                "type": "function",
                "function": {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.inputSchema,
                }
            }
            for tool in tools_result.tools
        ]
    
    async def process_query(self, query: str) -> str:
        """Process a query using OpenAI and MCP tools."""

        # get tools 
        tools = await self.get_mcp_tools()

        # initial openai response
        response = self.openai_client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": query
                }
            ],
            tools=tools,
            tool_choice="auto",
        )
        assistant_message = response.choices[0].message

        # initilize conversation with user query and assisant response
        messages = [
            {
                "role": "user",
                "content": query
            },
            assistant_message
        ]

        # handle tool calls if present
        if assistant_message.tool_calls:
            for tool_call in assistant_message.tool_calls:
                # execute tool call
                result = await self.session.call_tool(
                    name=tool_call.function.name,
                    arguments=json.loads(tool_call.function.arguments)
                )

                # add tool call to conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result.content[0].text,
                })
            
            # get the final response after tool calls
            final_response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=tools,
                tool_choice="auto",
            )

            return final_response.choices[0].message.content
        
        return assistant_message.content
    
    async def cleanup(self):
        """Cleanup the session and exit stack."""
        await self.exit_stack.aclose()

In [16]:
async def main():
    """Main function to run the MCP OpenAI client."""
    client = MCPOpenAIClient()
    await client.connect_to_server()

    query = query = "What is our company's vacation policy?"
    print(f"\nQuery: {query}")

    response = await client.process_query(query)
    print(f"\nResponse: {response}")

In [17]:
asyncio.run(main())


Connected to server with tools:
  - get_knowledge_base: Retrieve the entire knowledge base as a formatted string.

Returns:
    A formatted string containing all Q&A pairs from the knowledge base.


Query: What is our company's vacation policy?

Response: Our company's vacation policy is as follows:

- Full-time employees are entitled to 20 paid vacation days per year.
- Vacation days can be used after completing 6 months of employment.
- You may carry over up to 5 unused vacation days to the next year.
- Vacation requests must be submitted at least 2 weeks in advance through the HR portal.

Let me know if you need information on how to submit a request or have any other questions!
